## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-28-08-50-52 +0000,nypost,Utah mom Elleshia Seymour allegedly abducted h...,https://nypost.com/2025/12/28/us-news/utah-mom...
1,2025-12-28-08-24-45 +0000,nypost,Anonymous money fuels $5 million in attacks on...,https://nypost.com/2025/12/28/us-news/anonymou...
2,2025-12-28-08-18-25 +0000,nypost,Salvation Army bell ringer allegedly tries to ...,https://nypost.com/2025/12/28/us-news/florida-...
3,2025-12-28-08-02-05 +0000,nyt,"In Myanmar’s Election, ‘Voting Out of Fear, No...",https://www.nytimes.com/2025/12/28/world/asia/...
4,2025-12-28-07-37-08 +0000,bbc,War-torn Myanmar voting in widely criticised '...,https://www.bbc.com/news/articles/cy7vly6gm76o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
80,trump,12
256,new,11
133,year,7
41,war,6
172,before,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
121,2025-12-27-10-32-03 +0000,cbc,"Thailand, Cambodia sign new ceasefire deal to ...",https://www.cbc.ca/news/world/thailand-cambodi...,42
103,2025-12-27-12-45-38 +0000,wapo,"Thailand, Cambodia sign new ceasefire deal aft...",https://www.washingtonpost.com/world/2025/12/2...,36
45,2025-12-28-00-00-00 +0000,wsj,President Trump overrode some on the religious...,https://www.wsj.com/politics/policy/how-trump-...,34
63,2025-12-27-21-27-46 +0000,nypost,"Kim Jong Un hails bond of ‘blood, life and dea...",https://nypost.com/2025/12/27/world-news/kim-j...,33
111,2025-12-27-11-15-43 +0000,cbc,Zelenskyy meets with Carney in Halifax before ...,https://www.cbc.ca/news/politics/carney-zelens...,32


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
121,42,2025-12-27-10-32-03 +0000,cbc,"Thailand, Cambodia sign new ceasefire deal to ...",https://www.cbc.ca/news/world/thailand-cambodi...
45,34,2025-12-28-00-00-00 +0000,wsj,President Trump overrode some on the religious...,https://www.wsj.com/politics/policy/how-trump-...
26,29,2025-12-28-02-17-16 +0000,nypost,Heroic Texas father rescues abducted daughter ...,https://nypost.com/2025/12/27/us-news/heroic-t...
42,27,2025-12-28-00-11-17 +0000,cbc,"Amid a civil war, polls open in Myanmar for 1s...",https://www.cbc.ca/news/world/myanmar-election...
10,24,2025-12-28-05-33-21 +0000,nypost,"Australian cruise ship runs aground, strikes c...",https://nypost.com/2025/12/28/world-news/austr...
48,22,2025-12-27-23-31-16 +0000,nypost,Sicko teacher allegedly sent sexual messages t...,https://nypost.com/2025/12/27/us-news/sicko-te...
47,22,2025-12-27-23-42-39 +0000,nypost,Jewish group warned police of terror plot at H...,https://nypost.com/2025/12/27/world-news/jewis...
63,20,2025-12-27-21-27-46 +0000,nypost,"Kim Jong Un hails bond of ‘blood, life and dea...",https://nypost.com/2025/12/27/world-news/kim-j...
66,19,2025-12-27-20-57-25 +0000,nypost,"Iran declares ‘total war’ against US, Israel a...",https://nypost.com/2025/12/27/world-news/irani...
73,19,2025-12-27-19-46-16 +0000,cbc,"Jeffrey R. Holland, next in line to lead Churc...",https://www.cbc.ca/news/world/jeffrey-r-hollan...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
